<a href="https://colab.research.google.com/github/junca2306/psi/blob/main/Analise_FIDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fpdf
import pandas as pd
import requests
from fpdf import FPDF
import matplotlib.pyplot as plt
from datetime import datetime
import zipfile
import io
import os

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=7cb5ff23d8001a883cada51b88fa4082009cc9ecbbb619d6ff0c1aed6f5d073f
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
# ==============================================
# 1. COLETA DE DADOS DA CVM (FIDCs Cadastrados)
# ==============================================
def get_cvm_data():

    #url = "https://dados.cvm.gov.br/dados/FIDC/DOC/INF_MENSAL/DADOS/"
    url = "https://dados.cvm.gov.br/dados/FIDC/DOC/INF_MENSAL/DADOS/inf_mensal_fidc_202503.zip"
    #arquivo_zip = "inf_mensal_fidc_202503.zip"
    #url_arquivo = url + arquivo_zip
    destino_zip = "dados_fidc.zip"
    pasta_destino = "dados_fidc_extraidos"
    try:
        response = requests.get(url)
        #data = response.json()
        data = pd.read_csv(url)
        print("Dados da CVM carregados com sucesso.")
        return data
    except Exception as e:
        print(f"Erro ao acessar site da CVM: {e}")
        return None

In [ ]:
# ==============================================
# 2. CÁLCULOS DO FUNDO (Performance, INA5, etc.)
# ==============================================
def calculate_metrics(portfolio):

    df = pd.DataFrame(portfolio)

    # Performance média ponderada
    df['Retorno Ponderado'] = df['Performance (12m)'] * df['Peso no Portfólio'] / 100
    performance_total = df['Retorno Ponderado'].sum()

    # Cálculo do INA5 (exemplo simplificado)
    total_creditos_vigentes = 1_000_000  # Valor hipotético
    total_inadimplentes_5meses = 32_000   # Valor hipotético
    INA5 = (total_inadimplentes_5meses / total_creditos_vigentes) * 100

    return {
        "Performance Total": performance_total,
        "INA5": INA5,
        "Patrimonio": df['Peso no Portfólio'].sum() * 10_000_000  # Exemplo: 10 MM de PL
    }

In [ ]:
# ==============================================
# 3. GERAR CARTA MENSAL (PDF)
# ==============================================
def generate_pdf_report(metrics, url):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Cabeçalho
    pdf.cell(200, 10, txt="Carta Mensal - FIC FIDC [Nome do Fundo]", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Data: {datetime.now().strftime('%d/%m/%Y')}", ln=True, align='C')

    # Resumo Executivo
    pdf.cell(200, 10, txt="1. Resumo Executivo", ln=True, align='L')
    pdf.multi_cell(0, 10, txt=f"""
    - Performance do mês: {metrics['Performance Total']:.2f}%
    - Patrimônio líquido: R$ {metrics['Patrimonio']:,.2f}
    - Índice de Inadimplência (INA5): {metrics['INA5']:.2f}%
    """)

    # Portfólio (Tabela)
    pdf.cell(200, 10, txt="2. Portfólio do Fundo", ln=True, align='L')
    cols = ["FIDC", "Segmento", "Performance (12m)", "Peso no Portfólio"]
    for col in cols:
        pdf.cell(40, 10, txt=col, border=1)
    pdf.ln()
    for _, row in url.iterrows():
        pdf.cell(40, 10, txt=row['FIDC'], border=1)
        pdf.cell(40, 10, txt=row['Segmento'], border=1)
        pdf.cell(40, 10, txt=str(row['Performance (12m)']) + "%", border=1)
        pdf.cell(40, 10, txt=str(row['Peso no Portfólio']) + "%", border=1)
        pdf.ln()

    # Rodapé
    pdf.cell(100, 10, txt="Produzido por Juliana Neves Cassalho", ln=True, align='C')


    # Salvar PDF
    pdf.output("Carta_Mensal_FIC_FIDC.pdf")


    pdf.close()

    print("Carta mensal gerada: Carta_Mensal_FIC_FIDC.pdf")
    print("Produzido por Juliana Neves Cassalho - C136243")


In [ ]:
# ==============================================
# 4. GERAR PLANILHA (EXCEL)
# ==============================================
def generate_excel_report(metrics, portfolio):
    with pd.ExcelWriter("Calculos_FIC_FIDC.xlsx") as writer:
        # Planilha 1: Métricas
        df_metrics = pd.DataFrame({
            "Métrica": ["Performance Total", "INA5", "Patrimônio Líquido"],
            "Valor": [f"{metrics['Performance Total']:.2f}%",
                      f"{metrics['INA5']:.2f}%",
                      f"R$ {metrics['Patrimonio']:,.2f}"]
        })
        df_metrics.to_excel(writer, sheet_name="Métricas", index=False)

        # Planilha 2: Portfólio
        portfolio.to_excel(writer, sheet_name="Portfólio", index=False)

        # Planilha 3: Cálculo do INA5
        df_ina5 = pd.DataFrame({
            "Descrição": ["Total Créditos Vigentes", "Total Inadimplentes (5 meses)", "INA5"],
            "Valor": [1_000_000, 32_000, "=B3/B2*100"]
        })
        df_ina5.to_excel(writer, sheet_name="Cálculo INA5", index=False)

    print("Planilha gerada: Calculos_FIC_FIDC.xlsx")

In [ ]:
# ==============================================
# EXECUÇÃO PRINCIPAL
# ==============================================
if __name__ == "__main__":
    # Dados do portfólio (exemplo)
    portfolio_data = {
        "FIDC": ["FIDC Alpha", "FIDC Beta", "FIDC Gamma", "FIDC Delta", "FIDC Omega"],
        "Segmento": ["Varejo", "Saúde", "Energia", "Agronegócio", "Logística"],
        "Performance (12m)": [12.5, 10.8, 14.2, 9.5, 11.0],
        "Peso no Portfólio": [25, 20, 15, 25, 15]
    }
    df_portfolio = pd.DataFrame(portfolio_data)

    # Coleta de dados da CVM (simulada)
    cvm_data = get_cvm_data()  # Em produção, usar dados reais

    # Cálculos
    metrics = calculate_metrics(df_portfolio)

    # Gerar relatórios
    generate_pdf_report(metrics, df_portfolio)
    generate_excel_report(metrics, df_portfolio)

Erro ao acessar site da CVM: Multiple files found in ZIP file. Only one file per ZIP: ['inf_mensal_fidc_tab_III_202503.csv', 'inf_mensal_fidc_tab_II_202503.csv', 'inf_mensal_fidc_tab_IV_202503.csv', 'inf_mensal_fidc_tab_IX_202503.csv', 'inf_mensal_fidc_tab_I_202503.csv', 'inf_mensal_fidc_tab_VII_202503.csv', 'inf_mensal_fidc_tab_VI_202503.csv', 'inf_mensal_fidc_tab_V_202503.csv', 'inf_mensal_fidc_tab_X_1_1_202503.csv', 'inf_mensal_fidc_tab_X_1_202503.csv', 'inf_mensal_fidc_tab_X_202503.csv', 'inf_mensal_fidc_tab_X_2_202503.csv', 'inf_mensal_fidc_tab_X_3_202503.csv', 'inf_mensal_fidc_tab_X_4_202503.csv', 'inf_mensal_fidc_tab_X_5_202503.csv', 'inf_mensal_fidc_tab_X_6_202503.csv', 'inf_mensal_fidc_tab_X_7_202503.csv']
Carta mensal gerada: Carta_Mensal_FIC_FIDC.pdf
Produzido por Juliana Neves Cassalho - C136243
Planilha gerada: Calculos_FIC_FIDC.xlsx
